In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import random
import os
from datetime import datetime
# import torch
from stable_baselines import DQN
from stable_baselines import bench, logger
import tensorflow
import sys
sys.path.append('..')
from modules import utils, constants
from sklearn.model_selection import train_test_split
from stable_baselines.common.callbacks import CheckpointCallback
import matplotlib.pyplot as plt
%matplotlib inline

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Invalid MIT-MAGIC-COOKIE-1 key/home/lmuyama/anaconda3/envs/ml4hc_env/lib/python3.7/site-packages/stable_baselines/__init__.py:33: UserWarning: stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation.
  "stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation."


In [2]:
SEED = constants.SEED
random.seed(SEED)
np.random.seed(SEED)
# os.environ['PYTHONHASHSEED']=str(SEED)
tensorflow.set_random_seed(constants.SEED)
tensorflow.compat.v1.set_random_seed(constants.SEED)
# torch.manual_seed(SEED)
# torch.use_deterministic_algorithms(True)

#### The data

In [3]:
#train_df = pd.read_csv('../../final/data/train_set_noisy_6_missing_3.csv')
# train_val_df = pd.read_csv('../data/train_set_basic.csv')
# train_val_df = train_val_df.fillna(-1)
# train_val_df.head()

In [4]:
# X_train_val = train_val_df.iloc[:, 0:-1]
# y_train_val = train_val_df.iloc[:, -1]

In [5]:
# X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1, stratify=y_train_val, 
#                                                   random_state=SEED)

In [6]:
# train_df = pd.concat([X_train.reset_index(drop=True), y_train.reset_index(drop=True)], axis=1)
# val_df = pd.concat([X_val.reset_index(drop=True), y_val.reset_index(drop=True)], axis=1)

In [7]:
# train_df.to_csv('../data/train_set_basic.csv', index=False)
# val_df.to_csv('../data/val_set_constant.csv', index=False)

In [8]:
train_df = pd.read_csv('../data/train_set_basic.csv')
train_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,14.728733,-1.000000,3.170892,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,1,-1.000000,-1.000000,-1.0,-1.000000,-1.0,-1.000000,44.186200,-1.0,0
1,10.405752,9.634615,5.659537,-1.000000,-1.000000,77.413788,212.671838,4.032519,0,0.887130,96.311597,-1.0,43.218595,-1.0,83.207518,31.217256,-1.0,4
2,15.132737,358.914888,1.842252,3.797487,315.102272,80.500314,-1.000000,5.639507,0,-1.000000,-1.000000,-1.0,-1.000000,-1.0,-1.000000,45.398211,-1.0,0
3,11.340169,-1.000000,1.662209,2.441767,-1.000000,97.033963,102.079062,3.506041,1,1.020527,127.281715,-1.0,20.847013,-1.0,62.210273,34.020508,-1.0,6
4,6.691485,-1.000000,3.337971,-1.000000,-1.000000,99.838438,24.119564,2.010694,0,1.957666,34.633063,-1.0,34.612121,-1.0,112.411298,20.074456,-1.0,5


In [9]:
len(train_df)

50400

In [10]:
X_train = train_df.iloc[:, 0:-1]
y_train = train_df.iloc[:, -1]

X_train, y_train = np.array(X_train), np.array(y_train)
X_train.shape, y_train.shape

((50400, 17), (50400,))

#### The DQN Agent

In [ ]:
for steps in [int(6.5e6), int(7e6), int(7.5e6)]:
    dqn_model = utils.stable_dqn3(X_train, y_train, steps, True, f'../../final/models/testing/dqn_basic_{steps}')

In [11]:
training_env = utils.create_env(X_train, y_train)
training_env = bench.Monitor(training_env, logger.get_dir())
# model = DQN('MlpPolicy', training_env, verbose=1, seed=constants.SEED, learning_rate=0.0001, buffer_size=1000000, 
#             learning_starts=50000, train_freq=4, target_network_update_freq=10000, exploration_final_eps=0.05, 
#             n_cpu_tf_sess=1, policy_kwargs=dict(dueling=False), double_q=False)

model = DQN('MlpPolicy', training_env, verbose=1, seed=constants.SEED, learning_rate=0.0001, buffer_size=1000000, 
            learning_starts=50000, train_freq=4, target_network_update_freq=10000, exploration_final_eps=0.05, 
            n_cpu_tf_sess=1, policy_kwargs=dict(dueling=False), double_q=False)


checkpoint_callback = CheckpointCallback(save_freq=500000, save_path='../models/sb/dqn', 
                                         name_prefix='dqn_basic')

model.learn(total_timesteps=100000000, log_interval=500000, callback=checkpoint_callback)







Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where








--------------------------------------
| % time spent exploring  | 86       |
| episodes                | 500000   |
| mean 100 episode reward | -0.9     |
| steps                   | 1385725  |
| success rate            | 0.07     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 73       |
| episodes                | 1000000  |
| mean 100 episode reward | -0.8     |
| steps                   | 2821426  |
| success rate            | 0.15     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 59       |
| episodes                | 1500000  |
| mean 100 episode reward | -0.9     |
| steps                   | 4243342  |
| success rate    

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 8000000  |
| mean 100 episode reward | -0.8     |
| steps                   | 23111911 |
| success rate            | 0.13     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 8500000  |
| mean 100 episode reward | -0.9     |
| steps                   | 24468761 |
| success rate            | 0.18     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 9000000  |
| mean 100 episode reward | -0.8     |
| steps                   | 25865171 |
| success rate            | 0.12     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 9500000  |
| mean 100 episode reward | -1       |
| steps                  

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 23500000 |
| mean 100 episode reward | -0.8     |
| steps                   | 62371967 |
| success rate            | 0.14     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 24000000 |
| mean 100 episode reward | -0.8     |
| steps                   | 63394166 |
| success rate            | 0.11     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 24500000 |
| mean 100 episode reward | -0.8     |
| steps                   | 64436883 |
| success rate            | 0.13     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 25000000 |
| mean 100 episode reward | -0.8     |
| steps                  

KeyboardInterrupt: 

In [ ]:
# model.save('../models/vanilla_dqn_basic_anemia_diagnosis')